In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as layers
from tensorflow.keras import backend as K
import tensorflow_addons as tfa

from helper_functions import get_classification_report
from helper_functions import create_tf_data_datasets_contrastive
from helper_functions import create_tf_data_testset_contrastive
from helper_functions import euclidean_distance
from helper_functions import manhattan_distance
from helper_functions import contrastive_loss

import wandb
from wandb.keras import WandbCallback

In [43]:
class ConvNeXt_Block(layers.Layer):
    r""" ConvNeXt Block.
    Args:
        dim (int): Number of input channels.
        drop_path (float): Stochastic depth rate. Default: 0.0
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
    """

    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = layers.DepthwiseConv2D(kernel_size=7, padding='same')  # depthwise conv
        self.norm = layers.LayerNormalization(epsilon=1e-6)
        # pointwise/1x1 convs, implemented with linear layers
        self.pwconv1 = layers.Dense(4 * dim)
        self.act = layers.Activation('gelu')
        self.pwconv2 = layers.Dense(dim)
        self.drop_path = DropPath(drop_path)
        self.dim = dim
        self.layer_scale_init_value = layer_scale_init_value

    def build(self, input_shape):
        self.gamma = tf.Variable(
            initial_value=self.layer_scale_init_value * tf.ones((self.dim)),
            trainable=True,
            name='_gamma')
        self.built = True

    def call(self, x):
        input = x
        x = self.dwconv(x)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma * x

        x = input + self.drop_path(x)
        return x

class Downsample_Block(layers.Layer):
    """The Downsample Block in ConvNeXt
        Args:
            dim (int): number of channels
    """

    def __init__(self, dim):
        super().__init__()
        self.LN = layers.LayerNormalization(epsilon=1e-6)
        self.conv = layers.Conv2D(dim, kernel_size=2, strides=2)

    def build(self, input_shape):
        self.built = True

    def call(self, x):
        x = self.LN(x)
        x = self.conv(x)
        return x

class DropPath(tf.keras.layers.Layer):
    """The Drop path in ConvNeXt
        Reference:
            https://github.com/rishigami/Swin-Transformer-TF/blob/main/swintransformer/model.py
    """
    def __init__(self, drop_prob=None):
        super().__init__()
        self.drop_prob = drop_prob

    def call(self, x, training=None):
        return self._drop_path(x, self.drop_prob, training)


    def _drop_path(self, inputs, drop_prob, is_training):
        if (not is_training) or (drop_prob == 0.):
            return inputs

        # Compute keep_prob
        keep_prob = 1.0 - drop_prob

        # Compute drop_connect tensor
        random_tensor = keep_prob
        shape = (tf.shape(inputs)[0],) + (1,) * (len(tf.shape(inputs)) - 1)
        random_tensor += tf.random.uniform(shape, dtype=inputs.dtype)
        binary_tensor = tf.floor(random_tensor)
        output = tf.math.divide(inputs, keep_prob) * binary_tensor
        return output

In [44]:
def create_convnext_model(input_shape=(224, 224, 3), depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], emb_dim=1024, drop_path=0., layer_scale_init_value=1e-6):
    """ Function to construct the ConvNeXt Model

        Args:
            input_shape (tuple): (Width, Height , Channels)
            depths (list): a list of size 4. denoting each stage's depth
            dims (list): a list of size 4. denoting number of kernel's in each stage
            num_classes (int): the number of classes
            drop_path (float): Stochastic depth rate. Default: 0.0
            layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
        Returns:
            ConvNeXt model: an instance of tf.keras.Model
    """

    assert (len(depths) == 4 and len(dims) ==4), "Must provide exactly 4 depths and 4 dims"
    assert (len(input_shape) == 3), "Input shape must be (W, H, C)"

    input = layers.Input(shape=input_shape)

    # Stem + res2
    y = layers.Conv2D(dims[0], kernel_size=4, strides=4)(input)
    y = layers.LayerNormalization(epsilon=1e-6)(y)
    for i in range(depths[0]):
        y = ConvNeXt_Block(dims[0], drop_path, layer_scale_init_value)(y)

    # downsample + res3
    y = Downsample_Block(dims[1])(y)
    for i in range(depths[1]):
        y = ConvNeXt_Block(dims[1], drop_path, layer_scale_init_value)(y)

    # downsample + res4
    y = Downsample_Block(dims[2])(y)
    for i in range(depths[2]):
        y = ConvNeXt_Block(dims[2], drop_path, layer_scale_init_value)(y)

    # downsample + res5
    y = Downsample_Block(dims[3])(y)
    for i in range(depths[3]):
        y = ConvNeXt_Block(dims[3], drop_path, layer_scale_init_value)(y)

    y = layers.GlobalAveragePooling2D()(y)
    # final norm layer
    y = layers.LayerNormalization(epsilon=1e-6)(y)
    # Head
    y = layers.Dense(emb_dim, activation='relu')(y)

    return tf.keras.Model(inputs=input, outputs=y)

## 90K Rows RGB 1000 - Contrastive Eucledian

In [45]:
anchor_images_path = "npz_datasets/pairs_90k_224_224_rows_1000/anchor"
positive_images_path = "npz_datasets/pairs_90k_224_224_rows_1000/positive"
width, height, channels = 113, 113, 3
batch_size = 128
train_dataset, val_dataset = create_tf_data_datasets_contrastive(anchor_images_path, positive_images_path, batch_size, height, width, rgb=True)

anchor_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/anchor"
positive_images_test_path = "npz_datasets/test_pairs_90k_224_224_rows_1000/positive"
test_dataset = create_tf_data_testset_contrastive(anchor_images_test_path, positive_images_test_path, height, width, rgb=True)

In [49]:
run = wandb.init(project="Architecture_1",
                 config={"learning_rate": 0.004,
                         "weight_decay": 0.05,
                         "otimizer": "AdamW",
                         "loss_function": "contrastive loss",
                         "distance_function": "eucledian",
                         "epochs": 50,
                         "batch_size": 32,
                         "embedding_dimension": 1024,
                         "image_size": "113x113x3",
                         "architecture": "ConvNeXt - 90k - RGB"})

config = wandb.config

In [51]:
model = create_convnext_model(input_shape=(113, 113, 3), depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], emb_dim=config.embedding_dimension, drop_path=0., layer_scale_init_value=1e-6)

left_input = layers.Input(shape=(height, width, channels))
right_input = layers.Input(shape=(height, width, channels))

encoded_l = model(left_input)
encoded_r = model(right_input)

distance = layers.Lambda(euclidean_distance)([encoded_l, encoded_r])
prediction = layers.Dense(1, activation="sigmoid")(distance)

siamese_model = keras.models.Model([left_input, right_input], outputs=prediction)
#siamese_model.compile(loss=contrastive_loss, optimizer=tfa.optimizers.AdamW(weight_decay=config.weight_decay, learning_rate=config.learning_rate), metrics=["accuracy"])
siamese_model.compile(loss=contrastive_loss, optimizer="Adam", metrics=["accuracy"])

history_siamese_model = siamese_model.fit(train_dataset, epochs=config.epochs, validation_data=val_dataset, callbacks=[tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5),WandbCallback()])

Epoch 1/50
514/638 [=======================>......] - ETA: 45s - loss: 0.2502 - accuracy: 0.4948

KeyboardInterrupt: 

In [ ]:
run.finish()